<a href="https://colab.research.google.com/github/OlanreleOladeji/Demand-forecast--XGBoost/blob/main/Products_quantity_forecasting_model_(XGB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data upload and preprocessing

In [989]:
import pandas as pd

In [990]:
from google.colab import files

In [991]:
uploaded = files.upload()

Saving 1pharma-data.csv to 1pharma-data (3).csv


In [992]:
df = pd.read_csv('1pharma-data.csv')

In [993]:
df.head()

,Distributor,Customer Name,City,Country,Latitude,Longitude,Channel,Sub-channel,Product_ID,Product Class,Sales,Price,Revenue,Month,Year,Name of Sales Rep,Manager,Sales Team,Unnamed: 18
0,Gottlieb-Cruickshank,"Zieme, Doyle and Kunze",Lublin,Poland,51.2333,22.5667,Hospital,Private,Topipizole,Mood Stabilizers,4.0,368,1472.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
1,Gottlieb-Cruickshank,Feest PLC,Świecie,Poland,53.4167,18.4333,Pharmacy,Retail,Choriotrisin,Antibiotics,7.0,591,4137.0,January,2018,Jessica Smith,Britanny Bold,Delta,NaN
2,Gottlieb-Cruickshank,Medhurst-Beer Pharmaceutical Limited,Rybnik,Poland,50.0833,18.5000,Pharmacy,Institution,Acantaine,Antibiotics,30.0,66,1980.0,January,2018,Steve Pepple,Tracy Banks,Bravo,NaN
3,Gottlieb-Cruickshank,Barton Ltd Pharma Plc,Czeladź,Poland,50.3333,19.0833,Hospital,Private,Lioletine Refliruvax,Analgesics,6.0,435,2610.0,January,2018,Mary Gerrard,Britanny Bold,Delta,NaN
4,Gottlieb-Cruickshank,Keeling LLC Pharmacy,Olsztyn,Poland,53.7800,20.4942,Pharmacy,Retail,Oxymotroban Fexoformin,Analgesics,20.0,458,9160.0,January,2018,Anne Wu,Britanny Bold,Delta,NaN


Check dataset structure

In [994]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254082 entries, 0 to 254081
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Distributor        254082 non-null  object 
 1   Customer Name      254082 non-null  object 
 2   City               254082 non-null  object 
 3   Country            254082 non-null  object 
 4   Latitude           254082 non-null  float64
 5   Longitude          254082 non-null  float64
 6   Channel            254082 non-null  object 
 7   Sub-channel        254082 non-null  object 
 8   Product_ID         254082 non-null  object 
 9   Product Class      254082 non-null  object 
 10  Sales              254082 non-null  float64
 11  Price              254082 non-null  int64  
 12  Revenue            254082 non-null  float64
 13  Month              254082 non-null  object 
 14  Year               254082 non-null  int64  
 15  Name of Sales Rep  254082 non-null  object 
 16  Ma

Data Exploration and Cleaning

Print the column headings

In [995]:
print(df.columns.tolist())


['Distributor', 'Customer Name', 'City', 'Country', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product_ID', 'Product Class', 'Sales', 'Price', 'Revenue', 'Month', 'Year', 'Name of Sales Rep', 'Manager', 'Sales Team', 'Unnamed: 18']


Dropping columns

In [996]:
df= df.drop(['Channel', 'Sub-channel', 'Product Class', 'Name of Sales Rep', 'Price','Manager', 'Sales Team', 'Customer Name', 'City', 'Country', 'Distributor', 'Latitude', 'Longitude', 'Channel', 'Sub-channel', 'Product Class', 'Unnamed: 18'], axis=1)

In [997]:
df.head()

,Product_ID,Sales,Revenue,Month,Year
0,Topipizole,4.0,1472.0,January,2018
1,Choriotrisin,7.0,4137.0,January,2018
2,Acantaine,30.0,1980.0,January,2018
3,Lioletine Refliruvax,6.0,2610.0,January,2018
4,Oxymotroban Fexoformin,20.0,9160.0,January,2018


Sum the number of items in each month per year

In [998]:
monthly_sales = df.groupby(['Year', 'Month', 'Product_ID'])['Sales'].sum().reset_index()

In [999]:
Monthly_Revenue= df.groupby(['Year', 'Month', 'Product_ID'])['Revenue'].sum().reset_index()

In [1000]:
merged_df = pd.merge(monthly_sales, Monthly_Revenue, on=['Year', 'Month', 'Product_ID'])

In [1001]:
merged_df.head()

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0


In [1002]:
df= merged_df

In [1003]:
df

,Year,Month,Product_ID,Sales,Revenue
0,2017,April,Abatatriptan,1410.0,1046220.0
1,2017,April,Abilovir Aprotasol,1974.0,531006.0
2,2017,April,Abobozolid,1350.0,101250.0
3,2017,April,Abranatal Lysoprosate,3767.0,2565327.0
4,2017,April,Abtasol,594.0,447876.0
...,...,...,...,...,...
11515,2020,September,Zithrotrol,3609.0,494433.0
11516,2020,September,Zonitonin,1300.0,673400.0
11517,2020,September,Zonixamine Immulinum,435.0,129630.0
11518,2020,September,Zynosine,790.0,58460.0


In [1004]:
#print(df['Product_ID'].unique())

Feature Engineering

Split Dataset

In [1005]:
from sklearn.model_selection import train_test_split

In [1006]:
X = ['Revenue']
y = ['Sales']

In [1007]:
indices = df.index

In [1008]:
train_df, test_df = train_test_split(indices, test_size=0.2, random_state=42)


In [1009]:
train_df = df.loc[train_indices].copy()
test_df = df.loc[test_indices].copy()

In [1010]:
train_X = train_df[feature_cols]
train_y = train_df[target_col]
test_X = test_df[feature_cols]
test_y = test_df[target_col]

In [1011]:
train_df.shape

(9216, 5)

In [1012]:
test_df.shape

(2304, 5)

In [1013]:
display(train_df.head())

,Year,Month,Product_ID,Sales,Revenue
11473,2020,September,Sanctumine,1299.0,187591.0
782,2017,February,Bexridol,1471.0,117680.0
4080,2018,July,Abatatriptan,2474.0,1835708.0
1156,2017,January,Sevoderm,1322.0,876486.0
1536,2017,June,Dorzovorin Pentadarone,1067.0,174988.0


In [1014]:
display(test_df.head())

,Year,Month,Product_ID,Sales,Revenue
1078,2017,January,Halciramine,381.0,155829.0
9315,2020,December,Secrelazine Insonamic,932.0,646808.0
3299,2018,August,Pulmodiol Adalaxime,698.0,422290.0
7894,2019,May,Tetratanyl,5344.0,2730784.0
11037,2020,November,Zonixamine Immulinum,2365.0,704770.0


In [1015]:
train_X

,Revenue
11473,187591.0
782,117680.0
4080,1835708.0
1156,876486.0
1536,174988.0
...,...
11284,1305174.0
5191,404855.0
5390,3921854.0
860,350366.0


In [1016]:
train_y

,Sales
11473,1299.0
782,1471.0
4080,2474.0
1156,1322.0
1536,1067.0
...,...
11284,1731.0
5191,2165.0
5390,8563.0
860,1049.0


Dataset Scaling using MinMaxScaler

In [1017]:
test_X

,Revenue
1078,155829.0
9315,646808.0
3299,422290.0
7894,2730784.0
11037,704770.0
...,...
7033,494592.0
1833,2884492.0
5882,276422.0
8332,315360.0


In [1018]:
test_y

,Sales
1078,381.0
9315,932.0
3299,698.0
7894,5344.0
11037,2365.0
...,...
7033,1104.0
1833,5746.0
5882,3371.0
8332,1460.0


In [1019]:
from sklearn.preprocessing import MinMaxScaler

Scaling the train_X and test_X :

In [1020]:
scaler_X = MinMaxScaler()

In [1021]:
scaler_X.fit(train_X)

MinMaxScaler()

In [1022]:
train_X_Scaled = scaler_X.transform(train_X)

In [1023]:
train_X_Scaled = pd.DataFrame(train_X_Scaled, columns=feature_cols, index=train_X.index)

In [1024]:
train_X_Scaled.head()

,Revenue
11473,0.035393
782,0.034486
4080,0.056784
1156,0.044334
1536,0.035230


In [1025]:
test_X_Scaled = scaler_X.transform(test_X)

In [1026]:
test_X_Scaled = pd.DataFrame(test_X_Scaled, columns=feature_cols, index=test_X.index)

In [1027]:
test_X_Scaled.head()

,Revenue
1078,0.034981
9315,0.041353
3299,0.038439
7894,0.068402
11037,0.042106


Scaling the train_y and test_y :

In [1028]:
scaler_y = MinMaxScaler()

In [1029]:
scaler_y.fit(train_y)

MinMaxScaler()

In [1030]:
train_y_Scaled = scaler_y.transform(train_y)

In [1031]:
train_y_Scaled = pd.DataFrame(train_y_Scaled, columns=target_col, index=train_y.index)

In [1032]:
train_y_Scaled.head()

,Sales
11473,0.010522
782,0.011977
4080,0.020461
1156,0.010717
1536,0.008560


In [1033]:
test_y_Scaled = scaler_y.transform(test_y)

In [1034]:
test_y_Scaled = pd.DataFrame(test_y_Scaled, columns=target_col, index=test_y.index)

In [1035]:
test_y_Scaled.head(10)

,Sales
1078,0.002758
9315,0.007418
3299,0.005439
7894,0.044737
11037,0.019539
1142,0.006395
3833,0.005084
4205,0.042581
9120,0.022737
4978,0.020529


XGBoost Algorithm

In [1036]:
from xgboost import XGBRegressor, DMatrix

In [1037]:
model = XGBRegressor()

In [1038]:
model.fit(train_X_Scaled, train_y_Scaled)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Make predictions on the test dataset (test_X_Scaled):

In [1039]:
forecast = model.predict(test_X_Scaled)

In [1040]:
print(forecast)

[0.01352304 0.01482592 0.01230563 ... 0.01165497 0.0124406  0.01148146]


Summarize progress

In [1041]:
Progress_summary_df = pd.DataFrame({
        'Revenue': test_X_Scaled['Revenue'],
        'Actual': test_y_Scaled['Sales'],
        'forecast': forecast[:Actual.shape[0]].flatten()
})

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Revenue,Actual,forecast
1078,0.034981,0.002758,0.013523
9315,0.041353,0.007418,0.014826
3299,0.038439,0.005439,0.012306
7894,0.068402,0.044737,0.039625
11037,0.042106,0.019539,0.015432


Evaluate model with performance

In [1058]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

In [1059]:
mean_squared_error(test_y_Scaled, forecast)

0.0004245069285389036

In [1060]:
mean_absolute_error(test_y_Scaled, forecast)

0.008594419807195663

In [1061]:
rmse = np.sqrt(mean_squared_error(test_y_Scaled, forecast))
print("RMSE: %f" % (rmse))

RMSE: 0.020604


Inverse transformation

Transform prediction

In [1062]:
forecast_value = scaler_y.inverse_transform(forecast.reshape(-1, 1))

Transform revenue (test_X) to original values

In [1063]:
test_X = scaler_X.inverse_transform(test_X_Scaled)

In [1064]:
Revenue_value = pd.DataFrame(test_X, columns=['Revenue'])

Transform the test_y_Scaled back to the original scale (test_y)

In [1065]:
test_y = scaler_y.inverse_transform(test_y_Scaled)

In [1066]:
Actual_value = pd.DataFrame(test_y, columns=['Sales'])

Summary of the actual test_y and forecast value transformation

In [1068]:
Inverse_Summary_df = pd.DataFrame({
        'Actual_value': Actual_value['Sales'],
        'Revenue_value': Revenue_value['Revenue'],
        'forecast_value': forecast_value[:Actual_value.shape[0]].flatten()
})

print("Summary:")

display(Inverse_Summary_df.head(5))

Summary:


,Actual_value,Revenue_value,forecast_value
0,381.0,155829.0,1653.734009
1,932.0,646808.0,1807.764526
2,698.0,422290.0,1509.808594
3,5344.0,2730784.0,4739.577148
4,2365.0,704770.0,1879.421021


TRYING OUT WRMSSE

Calculating Root Mean Squared Scaled Error (RMSSE)

Forecast summary with product identifiers

In [1071]:
summary = Inverse_Summary_df.merge(
    test_df[['Year', 'Month', 'Product_ID']],
    left_index=True,
    right_index=True,
    how='left'
)

display(summary_with_identifiers.head())

,Actual_value,Revenue_value,forecast_value,Year,Month,Product_ID
1078,381.0,155829.0,1653.734009,2017,January,Halciramine
9315,932.0,646808.0,1807.764526,2020,December,Secrelazine Insonamic
3299,698.0,422290.0,1509.808594,2018,August,Pulmodiol Adalaxime
7894,5344.0,2730784.0,4739.577148,2019,May,Tetratanyl
11037,2365.0,704770.0,1879.421021,2020,November,Zonixamine Immulinum


In [886]:
def rmsse(train_y, test_y, forecast):
    forecast_mse = np.mean((Actual_value - forecast_value)**2, axis=1)
    train_mse = np.mean(np.diff(train_y)**2)

    return np.sqrt(forecast_mse / train_y_mse)

rmsse_vals = rmsse(train_y.filter(like="d_").values, Actual_value, forecast_value)
print(rmsse_vals)

0       0.259067
1       0.178263
2       0.165245
3       0.123031
4       0.098840
          ...   
2299    0.094633
2300    0.060636
2301    0.394506
2302    0.013387
2303    0.052389
Length: 2304, dtype: float64


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [1054]:
print('Number of values: %d' % len(rmsse_vals))
print('RMSSE mean: %.3f' % np.mean(rmsse_vals))

Number of values: 2304
RMSSE mean: 0.207


In [1072]:
Product_weights = test_df.groupby('Product_ID')['Revenue'].sum()

In [1073]:
Product_weights =  Product_weights / Product_weights.sum()

In [1074]:
print(Product_weights.head())

Product_ID
Abatatriptan             0.006487
Abilovir Aprotasol       0.004180
Abobozolid               0.000432
Abranatal Lysoprosate    0.004990
Abtasol                  0.009012
Name: Revenue, dtype: float64


In [1081]:
print('Number of weights: %s' % len(Product_weights))

Number of weights: 240


In [1077]:
sum_of_weights = Product_weights.sum()
print(sum_of_weights)

1.0


Hyperparemeter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    "learning_rate": [0.01, 0.1],
    "max_depth": [3, 7],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.1],
   }

In [ ]:
grid_search = GridSearchCV(estimator= XGBRegressor(), param_grid= param_grid, cv=5, verbose=1)

In [ ]:
grid_search.fit(train_X_Scaled, train_y_Scaled)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [3, 7],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000],
                         'subsample': [0.8, 1.0]},
             verbose=1)

Best parameters

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'subsample': 0.8}

Make predictions on the test dataset (test_X_Scaled) using grid search best estimator

In [ ]:
predictions_tuned = y_predict_tuned = grid_search.best_estimator_.predict(test_X_Scaled)

In [ ]:
print(predictions_tuned)

[0.06137752 0.06510728 0.06286702 ... 0.06199182 0.06261616 0.06199182]


Summarize progress after tunning

In [ ]:
Progress_summary_df = pd.DataFrame({
        'Actual': test_y_Scaled['Sales'],
        'Predicted_tuned': predictions_tuned[:test_y_Scaled.shape[0]]
    })

print("Progress summary:")

display(Progress_summary_df.head(5))

Progress summary:


,Actual,Predicted_tuned
0,0.053989,0.061378
1,0.058418,0.065107
2,0.056537,0.062867
3,0.093880,0.089279
4,0.069936,0.065894


Evaluate model with performance after tunning

In [ ]:
mean_squared_error(test_y_Scaled, predictions_tuned)

0.00038102251710370183

In [ ]:
mean_absolute_error(test_y_Scaled, predictions_tuned)

0.007939578965306282

In [ ]:
 root_mean_squared_error(test_y_Scaled, predictions_tuned)

0.019519798457622528